In [ ]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env file

from datasets import load_from_disk, Dataset, DatasetDict
import re
from copy import deepcopy
from tqdm.notebook import tqdm
from radon.metrics import mi_visit

from google import genai
from pydantic import BaseModel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
dataset = load_from_disk("mbpp_custom_dataset_with_correct_gemini_pairs")

In [16]:
df = pd.DataFrame(dataset)

In [17]:
# Add score_1 and score_2 columns, which are the scores obtained using radon's mi_visit function
df['score_1'] = df['code_output_1'].apply(lambda x: mi_visit(x, True))
df['score_2'] = df['code_output_2'].apply(lambda x: mi_visit(x, True))

<unknown>:7: SyntaxWarning: invalid escape sequence '\s'
<unknown>:8: SyntaxWarning: invalid escape sequence '\s'
<unknown>:8: SyntaxWarning: invalid escape sequence '\d'


In [ ]:
df 

,task_id,text,code,test_list,test_setup_code,challenge_test_list,code_output_1,code_output_2,score_1,score_2
0,603,Write a function to get a lucid number smaller...,def get_ludic(n):\r\n\tludics = []\r\n\tfor i ...,"[assert get_ludic(10) == [1, 2, 3, 5, 7], asse...",,[],"def get_ludic(n):\n """"""\n Generates Ludi...","def get_ludic(n):\n """"""\n Generates Ludi...",77.572492,83.302854
1,604,Write a function to reverse words in a given s...,def reverse_words(s):\r\n return ' '.jo...,"[assert reverse_words(""python program"")==(""pro...",,[],"def reverse_words(s):\n """"""\n Reverses t...","def reverse_words(s):\n """"""\n Reverses t...",58.416032,79.378853
2,605,Write a function to check if the given integer...,def prime_num(num):\r\n if num >=1:\r\n for...,"[assert prime_num(13)==True, assert prime_num(...",,[],"import math\n\ndef prime_num(num):\n """"""\n ...","def prime_num(num):\n """"""\n Checks if a ...",88.966135,94.314079
3,606,Write a function to convert degrees to radians...,import math\r\ndef radian_degree(degree):\r\n ...,"[assert radian_degree(90)==1.5707963267948966,...",,[],import math\n\ndef radian_degree(degree):\n ...,"def radian_degree(degree):\n """"""\n Conve...",100.000000,77.706728
4,608,Write a python function to find nth bell numbe...,def bell_Number(n): \r\n bell = [[0 for i i...,"[assert bell_Number(2) == 2, assert bell_Numbe...",,[],from math import comb\nfrom typing import Dict...,"def bell_Number(n: int) -> int:\n """"""\n ...",87.327715,78.998865
...,...,...,...,...,...,...,...,...,...,...
233,969,Write a function to join the tuples if they ha...,def join_tuples(test_list):\r\n res = []\r\n ...,"[assert join_tuples([(5, 6), (5, 7), (6, 8), (...",,[],from collections import defaultdict\n\ndef joi...,"def join_tuples(test_list):\n """"""\n Join...",93.951619,85.523452
234,970,Write a function to find minimum of two number...,"def min_of_two( x, y ):\r\n if x < y:\r\n ...","[assert min_of_two(10,20)==10, assert min_of_t...",,[],"def min_of_two(x, y):\n """"""\n Finds the mini...","def min_of_two(x, y):\n """"""\n Finds the mini...",92.558902,100.000000
235,971,Write a function to find the maximum number of...,"def maximum_segments(n, a, b, c) : \r\n\tdp = ...","[assert maximum_segments(7, 5, 2, 5) == 2, ass...",,[],"def maximum_segments(n, a, b, c):\n """"""\n ...","def maximum_segments(n, a, b, c):\n """"""\n ...",70.970860,64.888152
236,973,Write a python function to left rotate the str...,"def left_rotate(s,d):\r\n tmp = s[d : ] + s...","[assert left_rotate(""python"",2) == ""thonpy"" ...",,[],"def left_rotate(s, d):\n """"""\n Left rota...","import collections\n\ndef left_rotate(s, d):\n...",73.040668,80.112125


In [19]:
# Create a blank dataframe with prompt, chosen, and rejected columns
df_dpo = pd.DataFrame(columns=['prompt', 'chosen', 'rejected'])
df_dpo

,prompt,chosen,rejected


In [20]:
# Input the data into the dataframe
for index, row in df.iterrows():
    prompt = row['text']
    code_1 = row['code_output_1']
    code_2 = row['code_output_2']
    score_1 = row['score_1']
    score_2 = row['score_2']
    
    if score_1 >= score_2:
        chosen = code_1
        rejected = code_2
    else:
        chosen = code_2
        rejected = code_1

    chosen = f"```python\n{chosen}\n```"
    rejected = f"```python\n{rejected}\n```"
    
    df_dpo = pd.concat([df_dpo, pd.DataFrame({'prompt': [prompt], 'chosen': [chosen], 'rejected': [rejected]})], ignore_index=True)

In [21]:
df_dpo

,prompt,chosen,rejected
0,Write a function to get a lucid number smaller...,"```python\ndef get_ludic(n):\n """"""\n Gen...","```python\ndef get_ludic(n):\n """"""\n Gen..."
1,Write a function to reverse words in a given s...,"```python\ndef reverse_words(s):\n """"""\n ...","```python\ndef reverse_words(s):\n """"""\n ..."
2,Write a function to check if the given integer...,"```python\ndef prime_num(num):\n """"""\n C...",```python\nimport math\n\ndef prime_num(num):\...
3,Write a function to convert degrees to radians...,```python\nimport math\n\ndef radian_degree(de...,"```python\ndef radian_degree(degree):\n """"""..."
4,Write a python function to find nth bell numbe...,```python\nfrom math import comb\nfrom typing ...,```python\ndef bell_Number(n: int) -> int:\n ...
...,...,...,...
233,Write a function to join the tuples if they ha...,```python\nfrom collections import defaultdict...,"```python\ndef join_tuples(test_list):\n """"..."
234,Write a function to find minimum of two number...,"```python\ndef min_of_two(x, y):\n """"""\n Fin...","```python\ndef min_of_two(x, y):\n """"""\n Fin..."
235,Write a function to find the maximum number of...,"```python\ndef maximum_segments(n, a, b, c):\n...","```python\ndef maximum_segments(n, a, b, c):\n..."
236,Write a python function to left rotate the str...,```python\nimport collections\n\ndef left_rota...,"```python\ndef left_rotate(s, d):\n """"""\n ..."


In [22]:
final_dpo_dataset = Dataset.from_pandas(df_dpo)
final_dpo_dataset.save_to_disk("mbpp_custom_dataset_with_correct_gemini_pairs_dpo_ready")

Saving the dataset (0/1 shards):   0%|          | 0/238 [00:00<?, ? examples/s]

In [ ]:
def generate_prompt(prompt: str) -> list[dict]:
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a code generation model. Your task is to generate code snippets based on user prompts. You are to only write the full code for the required function and stop after returning the function output."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt},]
        },
    ]
    return messages

In [24]:
df_dpo_messages_prompt = pd.DataFrame(columns=['prompt', 'chosen', 'rejected'])

for index, row in df_dpo.iterrows():
    prompt = row['prompt']
    chosen = row['chosen']
    rejected = row['rejected']
    
    messages = generate_prompt(prompt)
    
    df_dpo_messages_prompt = pd.concat([df_dpo_messages_prompt, pd.DataFrame({'prompt': [messages], 'chosen': [chosen], 'rejected': [rejected]})], ignore_index=True)

In [25]:
df_dpo_messages_prompt['prompt'][0]

[{'role': 'system',
  'content': [{'type': 'text',
    'text': 'You are a code generation model. Your task is to generate code snippets based on user prompts. You are to only write the full code for the required function and no more.'}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': 'Write a function to get a lucid number smaller than or equal to n.\nThe main function is defined by the function signature: def get_ludic(n)\n'}]}]

In [26]:
final_dpo_dataset_with_messages = Dataset.from_pandas(df_dpo_messages_prompt)
final_dpo_dataset_with_messages.save_to_disk("mbpp_custom_dataset_with_correct_gemini_pairs_dpo_ready_with_messages")

Saving the dataset (0/1 shards):   0%|          | 0/238 [00:00<?, ? examples/s]